<a href="https://colab.research.google.com/github/SrivenkateswaraRao/Research-Project/blob/master/Pest_Scratch_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
from os import listdir
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
import tensorflow as tf 
import numpy as np


In [5]:
import keras as ks 

Using TensorFlow backend.


# import the necessary packages


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from google.colab.patches import cv2_imshow



Using TensorFlow backend.


In [0]:
from google.colab import files
files.upload() 

In [0]:
import CreatePestImageData as Imagedataset

In [5]:
file_list = drive.ListFile(
    {'q': "'1aXb25Wp9sG7mL6AQgYY9ElrGcERi95JU' in parents"}).GetList()

for f in file_list:
    print('title: %s, id: %s' % (f['title'], f['id']))
    fname = f['title']
    print('downloading to {}'.format(fname))
    f_ = drive.CreateFile({'id': f['id']})
    f_.GetContentFile(fname)

title: LABELS.zip, id: 1Rr_6cmS07z_hDZ1FVX6GBhUgLWC7cMS0
downloading to LABELS.zip
title: GREENPATROL_DB_V1.0.zip, id: 1LrZaY2eXwTII9KqoTj2ZF4cvOkJI9q7V
downloading to GREENPATROL_DB_V1.0.zip
title: predefined_classes.txt, id: 1srJJMul6750VCSL2-z9KrJ_QN9i6WK5Y
downloading to predefined_classes.txt


In [0]:
import CreatePestImageData as Imagedataset

In [0]:
download_Pests = drive.CreateFile({'id': '1LrZaY2eXwTII9KqoTj2ZF4cvOkJI9q7V'})


In [0]:
download_Annotations = drive.CreateFile({'id': '1Rr_6cmS07z_hDZ1FVX6GBhUgLWC7cMS0'})


In [0]:
!unzip LABELS.zip -d '/Pests'


In [0]:
!unzip GREENPATROL_DB_V1.0.zip -d '/Pests'


# Data PreProcessing and Transformation 

In [0]:
class_object = {0 : 'background' , 1 : 'egg_wf' ,
                2 : 'egg_bt', 3 : 'egg_ta', 4 : 'wf', 5 : 'bt', 6 : 'ta', 7 : 'ta_maps', 8 : 'ta_larvae', 9 : 'ta_maps+larvae', 10 : 'tomato'}

In [0]:
dataSet = Imagedataset.createImageDataset()

In [0]:
 dataSet.prepare_dataset('/Pests/LABELS' , class_object,   test_size=30)

In [0]:
(trainY , trainX) , (testY , testX ) = (dataSet.trainY , dataSet.trainX ) , (dataSet.testY , dataSet.testX )



In [0]:
trainX  = trainX.astype(int)
testY  = testY.astype(int)


In [0]:
batch_size =len(trainX)


In [44]:
print(trainX.shape, trainY.shape,testY.shape )


(16487, 32, 32, 3) (16487,) (30,)


In [0]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [0]:
trainX_scaled = scaler.fit_transform(trainX.astype(np.float64))


In [0]:
testX_scaled = scaler.fit_transform(testX.astype(np.float64))


In [0]:
trainX = trainX.reshape((-1, 32, 32, 1)) / 255.
testX = testX.reshape((-1, 32, 32, 1)) / 255.

In [0]:
# define the model
model_with_conv = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu, input_shape=(32, 32, 1)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Flatten(),    
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    #  10-node softmax layer, with each node representing a class of clothing.
    tf.keras.layers.Dense(10,  activation=tf.nn.softmax)
])
 
# compile the model
model_with_conv.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [66]:
model_with_conv.fit(trainX, trainY, epochs=5)
model_with_conv.evaluate(testX,  testY, verbose=2)

ValueError: ignored